# Normalization and Tuning Neural Networks - Lab

## Introduction

For this lab on initialization and optimization, let's look at a slightly different type of neural network. This time, we will not perform a classification task as we've done before (Santa vs not santa, bank complaint types), but we'll look at a linear regression problem.

We can just as well use deep learning networks for linear regression as for a classification problem. Do note that getting regression to work with neural networks is a hard problem because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case!

## Objectives
You will be able to:
* Build a nueral network using keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [73]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

## Loading the data

The data we'll be working with is data related to facebook posts published during the year of 2014 on the Facebook's page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set and delete any rows with missing data. Afterwards, briefly preview the data.

In [74]:
#Your code here; load the dataset and drop rows with missing values. Then preview the data.
df = pd.read_csv('dataset_Facebook.csv', sep=';')
df = df.dropna()
df.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


## Initialization

## Normalize the Input Data

Let's look at our input data. We'll use the 7 first columns as our predictors. We'll do the following two things:
- Normalize the continuous variables --> you can do this using `np.mean()` and `np.std()`
- Make dummy variables of the categorical variables (you can do this by using `pd.get_dummies`)

We only count "Category" and "Type" as categorical variables. Note that you can argue that "Post month", "Post Weekday" and "Post Hour" can also be considered categories, but we'll just treat them as being continuous for now.

You'll then use these to define X and Y. 

To summarize, X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category


Be sure to normalize your features by subtracting the mean and dividing by the standard deviation.  

Finally, y will simply be the "like" column.

In [75]:

X = df[['Page total likes', 'Post Month', 'Post Weekday', 'Post Hour', 'Paid', 'Type', 'Category']]
Y = df['like']
ohe = preprocessing.OneHotEncoder()
ss = StandardScaler()

enc_col_type = X['Type'].values.reshape(-1, 1)
encoder = ohe.fit(enc_col_type)
type_ohe = pd.DataFrame(encoder.transform(enc_col_type).toarray(),
                   columns=encoder.get_feature_names(['Type']))

enc_col_cat = X['Category'].values.reshape(-1, 1)
encoder = ohe.fit(enc_col_cat)
cat_ohe = pd.DataFrame(encoder.transform(enc_col_cat).toarray(),
                      columns=encoder.get_feature_names(['Category']))

X_s = pd.DataFrame(ss.fit_transform(X.drop(['Type', 'Category'], axis=1)), 
                   columns=X.drop(['Type', 'Category'], axis=1).columns)
X_full_s = pd.concat([X_s, cat_ohe,type_ohe], axis=1)
X_full_s.drop(['Category_1', 'Type_Link'], axis=1, inplace=True)
X_full_s.head()

/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Page total likes,Post Month,Post Weekday,Post Hour,Paid,Category_2,Category_3,Type_Photo,Type_Status,Type_Video
0,1.00496,1.506154,-0.065724,-1.105878,-0.62486,1.0,0.0,1.0,0.0,0.0
1,1.00496,1.506154,-0.558655,0.492065,-0.62486,1.0,0.0,0.0,1.0,0.0
2,1.00496,1.506154,-0.558655,-1.105878,-0.62486,0.0,1.0,1.0,0.0,0.0
3,1.00496,1.506154,-1.051585,0.492065,1.60036,1.0,0.0,1.0,0.0,0.0
4,1.00496,1.506154,-1.051585,-1.105878,-0.62486,1.0,0.0,1.0,0.0,0.0


In [76]:
X = X_full_s

In [77]:
X.head()

,Page total likes,Post Month,Post Weekday,Post Hour,Paid,Category_2,Category_3,Type_Photo,Type_Status,Type_Video
0,1.00496,1.506154,-0.065724,-1.105878,-0.62486,1.0,0.0,1.0,0.0,0.0
1,1.00496,1.506154,-0.558655,0.492065,-0.62486,1.0,0.0,0.0,1.0,0.0
2,1.00496,1.506154,-0.558655,-1.105878,-0.62486,0.0,1.0,1.0,0.0,0.0
3,1.00496,1.506154,-1.051585,0.492065,1.60036,1.0,0.0,1.0,0.0,0.0
4,1.00496,1.506154,-1.051585,-1.105878,-0.62486,1.0,0.0,1.0,0.0,0.0


Our data is fairly small. Let's just split the data up in a training set and a validation set!  The next three code blocks are all provided for you; have a quick review but not need to make edits!

In [78]:
#Code provided; defining training and validation sets
data_clean = pd.concat([X, Y], axis=1)
np.random.seed(123)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2)


In [79]:
#Code provided; building an initial model
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose=0)

In [80]:
#Code provided; previewing the loss through successive epochs
hist.history['loss'][:10]

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

Did you see what happend? all the values for training and validation loss are "nan". There could be several reasons for that, but as we already mentioned there is likely a vanishing or exploding gradient problem. recall that we normalized out inputs. But how about the outputs? Let's have a look.

In [81]:
y_train.head()

208     54.0
290     23.0
286     15.0
0       79.0
401    329.0
Name: like, dtype: float64

Yes, indeed. We didn't normalize them and we should, as they take pretty high values. Let
s rerun the model but make sure that the output is normalized as well!

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [82]:
Y = np.array(Y)
y = ss.fit_transform(Y.reshape(-1,1))

In [95]:
data_clean = pd.concat([X, pd.DataFrame(y)], axis=1)
np.random.seed(123)
train, validation = train_test_split(data_clean, test_size=0.2)

X_val = validation.iloc[:,0:10]
y_val = validation.iloc[:,10]
X_train = train.iloc[:,0:10]
y_train = train.iloc[:,10]

In [96]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. 
#(See our code snippet above!)

model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])
hist=model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val))

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 713us/step - loss: 1.3987 - mean_squared_error: 1.3987 - val_loss: 1.1758 - val_mean_squared_error: 1.1758
Epoch 2/100
396/396 [==============================] - 0s 76us/step - loss: 1.1264 - mean_squared_error: 1.1264 - val_loss: 1.1019 - val_mean_squared_error: 1.1019
Epoch 3/100
396/396 [==============================] - 0s 52us/step - loss: 1.0594 - mean_squared_error: 1.0594 - val_loss: 1.0667 - val_mean_squared_error: 1.0667
Epoch 4/100
396/396 [==============================] - 0s 54us/step - loss: 1.0232 - mean_squared_error: 1.0232 - val_loss: 1.0324 - val_mean_squared_error: 1.0324
Epoch 5/100
396/396 [==============================] - 0s 120us/step - loss: 1.0011 - mean_squared_error: 1.0011 - val_loss: 1.0210 - val_mean_squared_error: 1.0210
Epoch 6/100
396/396 [==============================] - 0s 72us/step - loss: 0.9889 - mean_squared_error: 0.9889 - val_loss: 1.0114 - 

396/396 [==============================] - 0s 43us/step - loss: 0.9103 - mean_squared_error: 0.9103 - val_loss: 0.9518 - val_mean_squared_error: 0.9518
Epoch 51/100
396/396 [==============================] - 0s 46us/step - loss: 0.9102 - mean_squared_error: 0.9102 - val_loss: 0.9450 - val_mean_squared_error: 0.9450
Epoch 52/100
396/396 [==============================] - 0s 73us/step - loss: 0.9070 - mean_squared_error: 0.9070 - val_loss: 0.9507 - val_mean_squared_error: 0.9507
Epoch 53/100
396/396 [==============================] - 0s 96us/step - loss: 0.9071 - mean_squared_error: 0.9071 - val_loss: 0.9505 - val_mean_squared_error: 0.9505
Epoch 54/100
396/396 [==============================] - 0s 94us/step - loss: 0.9078 - mean_squared_error: 0.9078 - val_loss: 0.9537 - val_mean_squared_error: 0.9537
Epoch 55/100
396/396 [==============================] - 0s 111us/step - loss: 0.9078 - mean_squared_error: 0.9078 - val_loss: 0.9554 - val_mean_squared_error: 0.9554
Epoch 56/100
396/396 [

Epoch 100/100
396/396 [==============================] - 0s 66us/step - loss: 0.8928 - mean_squared_error: 0.8928 - val_loss: 0.9457 - val_mean_squared_error: 0.9457


Finally, let's recheck our loss function. Not only should it be populated with numerical data as opposed to null values, but we also should expect to see the loss function decreasing with successive epochs, demonstrating optimization!

In [97]:
hist.history['loss'][:10]

[1.3987290208989924,
 1.1264215214083892,
 1.0593779738804308,
 1.0232075802003495,
 1.0010548839063356,
 0.9889441420032521,
 0.9779384768370426,
 0.9686586110278813,
 0.9625816005046921,
 0.9566941145393584]

Great! We have a converged model. With that, let's investigate how well the model performed with our good old friend, mean squarred error.

In [98]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)  

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

print("MSE_train:", MSE_train)
print("MSE_val:", MSE_val)

MSE_train: 0.886415367622916
MSE_val: 0.945747052113066


## Using Weight Initializers

##  He Initialization

Let's try and use a weight initializer. In the lecture, we've seen the He normalizer, which initializes the weight vector to have an average 0 and a variance of 2/n, with $n$ the number of features feeding into a layer.

In [101]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val),verbose=0)

In [103]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

In [104]:
print(MSE_train)
print(MSE_val)

0.9410365914887343
0.9287018846671385


The initializer does not really help us to decrease the MSE. We know that initializers can be particularly helpful in deeper networks, and our network isn't very deep. What if we use the `Lecun` initializer with a `tanh` activation?

## Lecun Initialization

In [105]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose=0)

In [107]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

In [108]:
print(MSE_train)
print(MSE_val)

0.92798852558739
0.9999079561945043


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impace of various optimization algorithms.

## RMSprop

In [109]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose = 0)

In [110]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

In [111]:
print(MSE_train)
print(MSE_val)

0.906794979010588
0.9116707523804706


## Adam

In [112]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose = 0)

In [113]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

In [114]:
print(MSE_train)
print(MSE_val)

0.9090509130489668
0.9102608891537122


## Learning Rate Decay with Momentum


In [25]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=10, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose = 0)

In [26]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

In [27]:
print(MSE_train)
print(MSE_val)

0.8188327426055082
0.9218409795298302


## Additional Resources
* https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Consumer_complaints.ipynb  

* https://catalog.data.gov/dataset/consumer-complaint-database  

* https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/  

* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/  

* https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/  

* https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network  

* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/


## Summary  

In this lab, we began to practice some of the concepts regarding normalization and optimization for neural networks. In the final lab for this section, you'll independently practice these concepts on your own in order to tune a model to predict individuals payments to loans.